In [4]:

import pandas as pd
from stocksymbol import StockSymbol
from fuzzywuzzy import process
import yfinance as yf



api_key = 'your_api_key_here'
ss = StockSymbol(api_key)



# symbol_list_dow = ss.get_symbol_list(index="DJI")
# print(type(symbol_list_dow))
# symbol_list_dow


# market_list = ss.market_list
# market_list



symbol_list_us = ss.get_symbol_list(market="us")
symbol_list_us


df_us = pd.DataFrame(symbol_list_us)
df_us['longName'] = df_us['longName'].str.upper()


df_us = df_us.head(500)
# 8780 records
print(df_us.shape)
print(df_us.columns)
print(df_us.head())


symbol_list_gb = ss.get_symbol_list(market="gb")
symbol_list_gb


df_gb = pd.DataFrame(symbol_list_gb)
df_gb['longName'] = df_gb['longName'].str.upper()


df_gb = df_gb.head(500)
# 5737 rows
print(df_gb.shape)
print(df_gb.columns)
print(df_gb.head())


(500, 6)
Index(['symbol', 'shortName', 'longName', 'exchange', 'market', 'quoteType'], dtype='object')
  symbol  shortName               longName exchange     market quoteType
0   AAPL      apple             APPLE INC.   NASDAQ  us_market    EQUITY
1   MSFT  microsoft  MICROSOFT CORPORATION   NASDAQ  us_market    EQUITY
2   GOOG   alphabet          ALPHABET INC.   NASDAQ  us_market    EQUITY
3  GOOGL   alphabet          ALPHABET INC.   NASDAQ  us_market    EQUITY
4   AMZN     amazon       AMAZON.COM, INC.   NASDAQ  us_market    EQUITY
(500, 6)
Index(['symbol', 'shortName', 'longName', 'exchange', 'market', 'quoteType'], dtype='object')
   symbol                        shortName                  longName exchange  \
0   TYT.L     TOYOTA MOTOR CORPORATION Y50  TOYOTA MOTOR CORPORATION      LSE   
1  RDSA.L  ROYAL DUTCH SHELL PLC 'A' ORD E     ROYAL DUTCH SHELL PLC      LSE   
2  RDSB.L  ROYAL DUTCH SHELL PLC 'B' ORD E     ROYAL DUTCH SHELL PLC      LSE   
3   AZN.L    ASTRAZENECA PLC ORD

In [5]:

symbol_list_us = ss.get_symbol_list(market="us")
symbol_list_us

# Let's convert all strings to Upper Case before we start to compare strings for siilarities and differences.
df_us = pd.DataFrame(symbol_list_us)
df_us['longName'] = df_us['longName'].str.upper()


# Take only the first 500 rows to make the matching run faster. FuzzyWuzzy tends to be pretty slow when the dataframes 
# get huge in size.
df_us = df_us.head(500)
# 8780 records
print(df_us.shape)
print(df_us.columns)
print(df_us.head())


symbol_list_gb = ss.get_symbol_list(market="gb")
symbol_list_gb


df_gb = pd.DataFrame(symbol_list_gb)
df_gb['longName'] = df_gb['longName'].str.upper()


# Take only the first 500 rows to make the matching run faster. FuzzyWuzzy tends to be pretty slow when the dataframes 
# get huge in size.
df_gb = df_gb.head(500)

print(df_gb.shape)
print(df_gb.columns)
print(df_gb.head())


(500, 6)
Index(['symbol', 'shortName', 'longName', 'exchange', 'market', 'quoteType'], dtype='object')
  symbol  shortName               longName exchange     market quoteType
0   AAPL      apple             APPLE INC.   NASDAQ  us_market    EQUITY
1   MSFT  microsoft  MICROSOFT CORPORATION   NASDAQ  us_market    EQUITY
2   GOOG   alphabet          ALPHABET INC.   NASDAQ  us_market    EQUITY
3  GOOGL   alphabet          ALPHABET INC.   NASDAQ  us_market    EQUITY
4   AMZN     amazon       AMAZON.COM, INC.   NASDAQ  us_market    EQUITY
(500, 6)
Index(['symbol', 'shortName', 'longName', 'exchange', 'market', 'quoteType'], dtype='object')
   symbol                        shortName                  longName exchange  \
0   TYT.L     TOYOTA MOTOR CORPORATION Y50  TOYOTA MOTOR CORPORATION      LSE   
1  RDSA.L  ROYAL DUTCH SHELL PLC 'A' ORD E     ROYAL DUTCH SHELL PLC      LSE   
2  RDSB.L  ROYAL DUTCH SHELL PLC 'B' ORD E     ROYAL DUTCH SHELL PLC      LSE   
3   AZN.L    ASTRAZENECA PLC ORD

In [6]:

# Merge based on the column with the same name in both dataframes
# 'inner' means it will keep only common rows
merged_df = pd.merge(df_us, df_gb, on='longName', how='inner')  

# 90 matches; 410 records DO NOT MATCH.
print(merged_df.shape)
merged_df.head()


(90, 11)


,symbol_x,shortName_x,longName,exchange_x,market_x,quoteType_x,symbol_y,shortName_y,exchange_y,market_y,quoteType_y
0,AAPL,apple,APPLE INC.,NASDAQ,us_market,EQUITY,0R2V.IL,APPLE INC APPLE ORD (CDI),IOB,gb_market,EQUITY
1,MSFT,microsoft,MICROSOFT CORPORATION,NASDAQ,us_market,EQUITY,0QYP.IL,MICROSOFT CORP MICROSOFT ORD (C,IOB,gb_market,EQUITY
2,GOOG,alphabet,ALPHABET INC.,NASDAQ,us_market,EQUITY,0HD6.IL,ALPHABET INC ALPHABET ORD CLAS,IOB,gb_market,EQUITY
3,GOOG,alphabet,ALPHABET INC.,NASDAQ,us_market,EQUITY,0RIH.IL,ALPHABET INC ALPHABET ORD CLAS,IOB,gb_market,EQUITY
4,GOOGL,alphabet,ALPHABET INC.,NASDAQ,us_market,EQUITY,0HD6.IL,ALPHABET INC ALPHABET ORD CLAS,IOB,gb_market,EQUITY


In [7]:

# We know quite a few records are getting dropped because in some instances the Company Name may be spelled slightly 
# different on the US stock exchange compared to how it is listed on the GB stock exchange. Let's use Fuzzy Wuzzy to 
# determine the closeness of the match between the Company Name column on on the different exchanges.


# Function to find the best match and score for a given longName in df_gb
def find_best_match(longName, df_gb):
    match, score, _ = process.extractOne(longName, df_gb['longName'])
    return pd.Series({'Matched_Name': match, 'Score': score})


# Apply the function to df_us and add the results to df_gb
df_us[['Matched_Name', 'Score']] = df_us['longName'].apply(find_best_match, df_gb=df_gb)


# Filter the rows with a minimum similarity score of 90% or greater.
merged_df = pd.merge(df_us[df_us['Score'] >= 90], df_gb, left_on='longName', right_on='longName', how='inner', suffixes=('_df_us', '_df_gb'))


# Display the final merged DataFrame
print(merged_df.columns)
print(merged_df.shape)
print(merged_df.head())


Index(['symbol_df_us', 'shortName_df_us', 'longName', 'exchange_df_us',
       'market_df_us', 'quoteType_df_us', 'Matched_Name', 'Score',
       'symbol_df_gb', 'shortName_df_gb', 'exchange_df_gb', 'market_df_gb',
       'quoteType_df_gb'],
      dtype='object')
(90, 13)
  symbol_df_us shortName_df_us               longName exchange_df_us  \
0         AAPL           apple             APPLE INC.         NASDAQ   
1         MSFT       microsoft  MICROSOFT CORPORATION         NASDAQ   
2         GOOG        alphabet          ALPHABET INC.         NASDAQ   
3         GOOG        alphabet          ALPHABET INC.         NASDAQ   
4        GOOGL        alphabet          ALPHABET INC.         NASDAQ   

  market_df_us quoteType_df_us           Matched_Name  Score symbol_df_gb  \
0    us_market          EQUITY             APPLE INC.    100      0R2V.IL   
1    us_market          EQUITY  MICROSOFT CORPORATION    100      0QYP.IL   
2    us_market          EQUITY          ALPHABET INC.    100   

In [8]:

# Function to get stock prices for a given symbol and a given data range (in this example....one day)
def get_stock_prices(symbol):
    try:
        stock_data = yf.download(symbol, start='2022-02-01', end='2022-02-02')  # Adjust the date range as needed
        return stock_data['Close'].tolist()  # Assuming you want to get the closing prices
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Apply the function to each row in 'symbol_df_us' and create a new column 'price_us'
merged_df['price_us'] = merged_df['symbol_df_us'].apply(get_stock_prices)

# Apply the function to each row in 'symbol_df_gb' and create a new column 'price_gb'
merged_df['price_gb'] = merged_df['symbol_df_gb'].apply(get_stock_prices)


pd.options.display.max_columns = None
pd.options.display.max_rows = None
print(merged_df.head(10))


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['FB']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['BRK.A']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

...


[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['0A2R.IL']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******


1 Failed download:
['0A2E.IL']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

In [3]:

# Now, obviously, these are end-of-day-prices, and not prices based on real-time data, so there are no immediate arbitrage 
# opportunities available here, but it's a fun exercise nonetheless.


# There can be several reasons for price discrepancies between the US and UK stock prices for the same stock:

#  Currency Exchange Rates: The most common reason is the difference in currency. Prices are quoted in different currencies 
#  in the US and the UK. Exchange rates can fluctuate, leading to variations in the reported prices when converted.

#  Market Hours: Stock markets in different countries have different trading hours. If you're comparing prices during 
#  non-overlapping trading hours, you may see differences. Additionally, pre-market and after-hours trading can also 
#  contribute to variations.

#  Market Conditions: Market conditions, liquidity, and trading volumes can vary between the US and UK markets. Low 
#  liquidity can result in wider bid-ask spreads, leading to price differences.

#  Data Sources: Different data sources may provide slightly different prices due to variations in data feeds, reporting 
#  times, and data quality.

#  Dividends and Corporate Actions: Differences in dividend payments, stock splits, and other corporate actions can also 
#  affect stock prices differently in different markets.

#  Arbitrage and Trading Strategies: Traders and algorithms may engage in arbitrage or other trading strategies to exploit 
#  price differences between markets, which can temporarily widen the gap.

#  Regulatory Differences: Regulatory requirements and reporting standards can differ between countries, influencing how 
#  stock prices are calculated and reported.


# It's essential to consider these factors and, if needed, normalize the data or compare prices using a common currency to 
# get a more accurate comparison. Keep in mind that financial markets are dynamic, and prices can change rapidly based on 
# various factors.


# API Key Here:
# https://stock-symbol.herokuapp.com/
# https://stock-symbol.herokuapp.com/market_index_list

# https://pypi.org/project/stocksymbol/
# https://algotrading101.com/learn/yfinance-guide/

